In [1]:
import os
import argparse
import time
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

In [2]:
from PlotSpinalCordDatafile_withSlidingWindow import test as sodiumInterpVals
from PlotSpinalCordDatafile_withSlidingWindow import interpTimes, times

from PlotSpinalCordDatafile_withSlidingWindow import dataArr
from PlotSpinalCordDatafile_withSlidingWindow import slidingData 


BUN , BUN/Creatinine Ratio
0.8774417368026577
BUN/Creatinine Ratio , BUN
0.8774417368026577


In [22]:
method = 'dopri5'   #
learning_rate = 1e-3
data_size = 26
# train_len = data_size
train_len = int(data_size * 0.8)
test_len = int((data_size - train_len))
# valid_len = data_size - train_len - test_len
batch_size = 8
batch_time = test_len-1 # Must be in [1, test_len-1]
niters = 1000 # epoch/ number of iteration steps
input_dim = 2
output_dim = 1
hidden_dim1 = 60
hidden_dim2 = 60
data_index = 0 # Biomarker index
gpu = 0
adjoint = False
print(train_len)

20


In [4]:
if adjoint:
    from torchdiffeq import odeint_adjoint as odeint
    # print('adjoint is:odeint_adjoint')
else:
    from torchdiffeq import odeint
    # print('ajoint is odeint')

In [5]:
device = torch.device('cuda:' + str(args.gpu) if torch.cuda.is_available() else 'cpu')

In [6]:
print(dataArr[0])
print(times)

[141. 140. 138. 139. 138. 141. 139. 141. 142. 139. 142. 139. 142. 142.
 140. 139. 139. 141. 137. 141. 140. 141. 144. 145. 147. 143.]
[1217737. 1016080.  874834.  834544.  774086.  733753.  693271.  632994.
  566848.  473272.  432863.  392559.  372264.  352148.  321956.  301766.
  271567.  239827.  170814.  130406.   98711.   71492.    7721.    6179.
    5585.       0.]


In [7]:
times = (np.flip(times))/24/60
print(times)

[  0.           3.87847222   4.29097222   5.36180556  49.64722222
  68.54930556  90.55972222 118.62083333 166.54652778 188.58819444
 209.55972222 223.58055556 244.54722222 258.51666667 272.61041667
 300.59930556 328.66111111 393.64444444 439.57916667 481.43819444
 509.55069444 537.55972222 579.54444444 607.52361111 705.61111111
 845.65069444]


In [8]:
def moving_average(data):
    noStr = np.flip(data) #define noStr to be the sodium 
    window_size = 3
    moving_averages = [] #array to keep the new "long wave" values

    ##Checking to see if the variables are defined properly
    #print(len(noStr))
    #print(len(np.flip(times)))

    # Loop through the array t o
    #consider every window of size 3
    i = 0
    while  i < len(noStr)-window_size+1:

        # Calculate the average of current window
        window_average = round(np.sum(noStr[i:i+window_size]) / window_size, 2)

        # Store the average of current
        # window in moving average list
        moving_averages.append(window_average)

        # Shift window to right by one position
        i += 1

    #print(noStr[23:26]) # pulling from the orginal data to identify the 2 values we need to solve for < t_n -> t_n-2 >   
    moving_averages.append(noStr[24]) # First assign f(t_24) from no
    moving_averages.append(noStr[25])
    moving_averages[24] = round((moving_averages[23]+moving_averages[25])/2,2)
    return(np.asarray(moving_averages))

In [9]:
window_avgs = moving_average(dataArr[1])
print(window_avgs)

[3.67 3.83 3.9  4.13 4.13 4.3  4.23 4.17 4.1  4.3  4.33 4.27 4.2  4.2
 4.1  4.03 4.03 4.2  4.13 3.9  3.77 3.83 3.97 4.   4.   4.  ]


In [10]:
print(times)

[  0.           3.87847222   4.29097222   5.36180556  49.64722222
  68.54930556  90.55972222 118.62083333 166.54652778 188.58819444
 209.55972222 223.58055556 244.54722222 258.51666667 272.61041667
 300.59930556 328.66111111 393.64444444 439.57916667 481.43819444
 509.55069444 537.55972222 579.54444444 607.52361111 705.61111111
 845.65069444]


In [11]:
print(slidingData[0]) 

[145.0, 145.33, 143.33, 141.67, 140.67, 139.33, 139.67, 139.0, 139.67, 139.33, 140.33, 141.33, 141.0, 141.0, 140.0, 141.0, 140.67, 140.67, 140.33, 139.33, 139.33, 138.33, 139.0, 139.67, 140.33, 141.0]


In [12]:
true_y = torch.tensor(window_avgs.copy().astype('float32')).reshape((-1,1)).to(device)
t = torch.tensor(np.flip(times/24/60).copy()).to(device)
print(true_y)
best_loss = 1000000000

tensor([[3.6700],
        [3.8300],
        [3.9000],
        [4.1300],
        [4.1300],
        [4.3000],
        [4.2300],
        [4.1700],
        [4.1000],
        [4.3000],
        [4.3300],
        [4.2700],
        [4.2000],
        [4.2000],
        [4.1000],
        [4.0300],
        [4.0300],
        [4.2000],
        [4.1300],
        [3.9000],
        [3.7700],
        [3.8300],
        [3.9700],
        [4.0000],
        [4.0000],
        [4.0000]])


In [13]:
#Learnable ML data 

def create_dataset(dataset, time_set, batch_time, len):
    dataY0, dataY , time_T = [], [], []
    for i in range(len-batch_time):
        dataY0.append(dataset[i]) #Initail values for ODE
        dataY.append(dataset[i:(i+batch_time)]) #predict batch_time, steps/label ahead from initial
        time_T.append(time_set[i:(i+batch_time)])
    return dataY0, dataY, time_T


In [14]:
#Training 
train_y0, train_y, train_t = create_dataset(dataset = true_y[:train_len], time_set = t[:train_len], batch_time = batch_time , len = train_len)
train_y0 = torch.stack(train_y0)  # (M, 1, D)
train_y = torch.stack(train_y) # (M, T, 1, D)
train_t = torch.stack(train_t) # (M, T)
print(train_y0.shape)
print(train_y.shape)
print(train_t.shape)

torch.Size([15, 1])
torch.Size([15, 5, 1])
torch.Size([15, 5])


In [15]:
#Testing
test_y0, test_y, test_t = create_dataset(dataset=true_y[train_len:(train_len+test_len)], time_set=t[train_len:(train_len+test_len)],
                                              batch_time=batch_time, len=test_len)
test_y0 = torch.stack(test_y0)
test_y = torch.stack(test_y)
test_t = torch.stack(test_t) 
print(test_y0.shape)
print(test_y.shape)
print(test_t.shape)

torch.Size([1, 1])
torch.Size([1, 5, 1])
torch.Size([1, 5])


In [16]:
# ------------ Put data and labels together via TensorDataset -------------

train_dataset = torch.utils.data.TensorDataset(train_y0, train_y, train_t)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(test_y0, test_t, test_t)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [17]:
class ODEFunc(nn.Module):

    def __init__(self):
        super(ODEFunc, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.Tanh(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.Tanh(),
            nn.Linear(hidden_dim2, output_dim)
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        y = torch.squeeze(y)
        y_new = torch.hstack((t, y))
        y_new = y_new[None, None, :]
        #print(f'new y is: {y_new}, {y_new.shape}')
        return self.net(y_new)

In [18]:
train_y.shape

torch.Size([15, 5, 1])

In [19]:
ii = 0  #
func = ODEFunc().to(device)  # RHS of ODE system parameterized by FNN
optimizer = optim.Adam(func.parameters(), lr=learning_rate)
start = time.time()
best_loss = 10000000
train_loss_vector = []   # batch loss on training set
test_loss_vector = []    # total loss on test set
best_iter = 0
x=0
y=0

for itr in range(1, niters + 1):  # iteration starts from 1
    test_loss = 0.0
    train_loss = 0.0
    for batch_idx, (y0_train, y_train, t_train) in enumerate(train_loader):  # mini-batch
        batch_loss = 0.0
        for i in range(len(t_train)):  # feed the samples from mini-batch to odeint one by one
            # print(f'------ batch idx {batch_idx}, i is: {i}, len of t is: {len(t_train)}')
            y_train1 = y_train[i]  # pick one sample with shape (T, 1, D)
            y_train1 = y_train1[:, None, None, :]  # convert shape of y from (T, 1, D) to ideal shape (T, 1, 1, D)
            y0_train1 = y0_train[i]  # pick one initial value with shape (1, D)
            y0_train1 = y0_train1[None, :]  # convert shape of y0 from (1, D) to ideal shape (1, 1, D)
            pred_y = odeint(func, y0_train1, t_train[i]).to(device)  # shape of t_train is (batch_time, )
            loss = torch.mean(torch.square(pred_y - y_train1))
            batch_loss += loss
        optimizer.zero_grad()   # zero all your gradient before you do the backpro.
        batch_loss.backward()   # backpro to compute the gradient
        optimizer.step()  # update the parameters theta=(W,b) using the gradient obtained using loss.backward()
        train_loss += batch_loss.item()  # count training loss for all training data points
        train_loss_vector.append(train_loss)
    with torch.no_grad():
        for batch_idx1, (y0_test, y_test, t_test) in enumerate(test_loader):  # mini-batch
            batch_loss1 = 0.0
            for i in range(len(t_test)):  # feed samples from mini-batch to odeint one by one
                y_test1 = y_test[i]  # pick one element with shape (T, 1, D)
                #y_test1 = y_test1[:, None, 1, :]  # convert the shape from (T, 1, D) to (T, 1, 1, D)
                y0_test1 = y0_test[i]  # pick one initial with shape (1, D)
                y0_test1 = y0_test1[None, :]   # convert the shape to (1, 1, D)
                pred_y1 = odeint(func, y0_test1, t_test[i])
                #print(pred_y)
                loss1 = torch.mean(torch.square(pred_y1 - y_test1))
                batch_loss1 += loss1.item()
            test_loss += batch_loss1
            test_loss_vector.append(test_loss)
        if test_loss < best_loss:
            best_loss = test_loss
            best_iter = itr
            save_path = f'/Users/dezmon/CSCE587/REU/best_model_{batch_time}_{niters}.pth'
            torch.save(func.state_dict(), save_path)  # saving the model
            
end = time.time()
print(f'training time is {end-start}')
_bt = batch_time
_bs = batch_size
while(os.path.exists(f'/Users/dezmon/CSCE587/REU/training loss_{_bt}_{_bs}.txt')):
    _bt += 1
    _bs += 1
                
#np.savetxt(f'training loss_{batch_time}_{batch_size}.txt', train_loss_vector, delimiter='\n')
#np.savetxt(f'val loss_{batch_time}_{batch_size}.txt', test_loss_vector, delimiter='\n')


training time is 158.087060213089


In [20]:
# ----------- Visualization --------------
func.load_state_dict(
    torch.load(f'/Users/dezmon/CSCE587/REU/best_model_/best_model_{x}_{y}.pth', map_location=lambda storage, loc: storage))
# -------- short term prediction ------------
start_point = 0
end_point = 2
l = 0
D = 1  # dim of bio-markers
y_pred = np.zeros((data_size, D))
N = np.floor(data_size/end_point)  # - 1
for i in range(int(N)):
    a = true_y[start_point]  # pick up the initial value
    a = a[None, :]  # convert dim to (1, 1, D)
    t0 = t[start_point:start_point + end_point]  # (T, )
    b = odeint(func, a, t0)  # shape (T, 1, 1, D)
    b = b.detach().cpu().numpy()
    b = np.reshape(b, (end_point, D))  # shape (T, D) with T=end_point
    print(f'i {i} and start {start_point}')
    y_pred[start_point: start_point+end_point] = b
    start_point += end_point

np.savetxt(f'final prediction_{end_point}-steps.txt', y_pred, delimiter='\n')
plt.plot(t.cpu(), true_y[:, 0].cpu(), '-o')
plt.plot(t.cpu(), y_pred[:, 0], '-*')
plt.xlabel('t')
plt.ylabel('y')
plt.legend(['ground truth', 'prediction'])
plt.title(f'pred_{end_point}-steps_{batch_time}_{niters}')
plt.show()

# Relative Error
relative_error = torch.sum(torch.abs(torch.tensor(y_pred[:,0]) - true_y[:, 0].cpu()) / true_y[:, 0].cpu())
print(f'Relative Error: {relative_error.item()}')


NameError: name 'x' is not defined